In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, TimeDistributed, LSTM, Dense, Flatten, Dropout
from keras.utils import to_categorical
from keras import metrics
import tensorflow as tf

2025-07-08 11:27:26.394375: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-08 11:27:26.405138: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751963246.416962   29923 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751963246.420461   29923 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751963246.430000   29923 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
# Check whether tensorflow sees the GPU
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
# Load the data
X = np.load("../../mel_spectrograms_train.npy")
y = np.load("../../labels_train.npy")

In [4]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

In [5]:
# === CNN + LSTM Model ===
model = Sequential([
    TimeDistributed(Conv2D(16, (3, 3), activation='relu', padding='same')),
    TimeDistributed(MaxPooling2D((2, 2))),
    TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same')),
    TimeDistributed(MaxPooling2D((2, 2))),
    TimeDistributed(Flatten()),
    Dropout(0.1),
    LSTM(512, dropout=0.2, recurrent_dropout=0.1),
    Dropout(0.1),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

# === Training ===
model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=10, batch_size=8)

model.save('model.keras')

I0000 00:00:1751963249.267661   29923 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5588 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6


Epoch 1/10


I0000 00:00:1751963255.817243   29990 cuda_dnn.cc:529] Loaded cuDNN version 90501


1495/1495 ━━━━━━━━━━━━━━━━━━━━ 83s 51ms/step - accuracy: 0.8125 - loss: 0.4046 - val_accuracy: 0.9555 - val_loss: 0.1172
Epoch 2/10
1495/1495 ━━━━━━━━━━━━━━━━━━━━ 75s 50ms/step - accuracy: 0.9475 - loss: 0.1468 - val_accuracy: 0.9645 - val_loss: 0.0982
Epoch 3/10
1495/1495 ━━━━━━━━━━━━━━━━━━━━ 75s 50ms/step - accuracy: 0.9612 - loss: 0.1025 - val_accuracy: 0.9779 - val_loss: 0.0557
Epoch 4/10
1495/1495 ━━━━━━━━━━━━━━━━━━━━ 75s 50ms/step - accuracy: 0.9778 - loss: 0.0683 - val_accuracy: 0.9793 - val_loss: 0.0596
Epoch 5/10
1495/1495 ━━━━━━━━━━━━━━━━━━━━ 75s 50ms/step - accuracy: 0.9843 - loss: 0.0480 - val_accuracy: 0.9836 - val_loss: 0.0546
Epoch 6/10
1495/1495 ━━━━━━━━━━━━━━━━━━━━ 75s 50ms/step - accuracy: 0.9852 - loss: 0.0442 - val_accuracy: 0.9866 - val_loss: 0.0485
Epoch 7/10
1495/1495 ━━━━━━━━━━━━━━━━━━━━ 75s 50ms/step - accuracy: 0.9874 - loss: 0.0393 - val_accuracy: 0.9870 - val_loss: 0.0317
Epoch 8/10
1495/1495 ━━━━━━━━━━━━━━━━━━━━ 76s 51ms/step - accuracy: 0.9928 - loss: 0.02